# Render de modelos 3D en Realidad Aumentada con Pyrender

[Pyrender](https://pyrender.readthedocs.io/en/latest/index.html) es una biblioteca de Python para el render de escenas 3D que implementa [PBR (Physically Based Rendering)](https://en.wikipedia.org/wiki/Physically_based_rendering) y capaz de interpretar modelos en formato [glTF 2.0](https://www.khronos.org/gltf/) propuesto por el grupo [Khronos](https://www.khronos.org/), responsable de múltiples estándares abiertos.

Pyrender actualmente presenta una ventaja y un inconveniente destacados:
* Ventaja. Ofrece una API muy simple de usar para la carga y render de modelos.
* Inconveniente. Su desarrollo y mantenimiento lleva unos años detenido por lo que la calidad del render de los modelos es muy básica y presenta algunos bugs.

Para la carga de los modelos Pyrender hace uso de la biblioteca [trimesh](https://pypi.org/project/trimesh/) para la carga y procesado de modelos de mallas de triángulos.

In [1]:
import pyrender
import trimesh

El renderizado de una escena 3D necesita:
* Uno o varios modelos 3D.
* Una o varias luces, imprescindibles para que se pueda ver el modelo. Una de las luces puede ser una luz ambiental omnipresente que no necesita que se indique su ubicación.
* Una cámara en la que se hará la proyección de la escena.

Modelos, luces y cámara deben estar ubicados en la escena en ubicaciones y posiciones relativas a un origen de coordenadas. En Realidad Aumentada, el origen de coordenadas de la escena lo ubicaremos en el centro del marcador Aruco y la cámara estará en la posición relativa de la webcam con respecto al marcador.

In [2]:
import cv2
import numpy as np
import camara
import cuia

La localización, rotación y escala de los elementos de la escena 3D se especifica mediante [matrices de transformación](https://es.wikipedia.org/wiki/Matriz_de_transformaci%C3%B3n), que representan dichas operaciones en forma de matrices 4x4. Para la creación y composición de dichas matrices podemos apoyarnos en la biblioteca [mathutils](https://docs.blender.org/api/blender_python_api_current/mathutils.html), originada en el proyecto [Blender](https://www.blender.org/) para ofrecer funciones y tipos de datos que ayuden a representar elementos y operaciones necesarios en geometría 3D.

In [3]:
import mathutils
import math

**Nota**: el sistema de coordenadas usado por Pyrender (el mismo que usa OpenGL) tiene una orientación distinta al que usa OpenCV. 

![Sistems de coordenadas de OpenCV y Pyrender](media/opencvpyrender.png "Sistems de coordenadas de OpenCV y Pyrender")

Por ello será necesario hacer la conversión adecuada para el uso combinado de OpenCV y Pyrender.

Dada la pose percibida del marcador y calculada mediante [solvePnP](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga549c2075fac14829ff4a58bc931c033d), que tendremos especificada en forma de un vector de translación **tvec** y un vector de rotación **rvec**, necesitamos expresarla en forma de matriz de transformación y adaptarla al modelo de sistema de coordenadas de Pyrender. Esto implicará el uso de la función [Rodrigues](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga61585db663d9da06b68e70cfbf6a1eac) y la inversión de las componentes Y y Z.

In [4]:
def fromOpencvToPyrender(rvec, tvec):
    pose = np.eye(4)
    pose[0:3,3] = tvec.T
    pose[0:3,0:3] = cv2.Rodrigues(rvec)[0]
    pose[[1,2]] *= -1
    pose = np.linalg.inv(pose)
    return(pose)

Esta función será la que nos indique la matriz de transformación empleada para ubicar la cámara en Pyrender (en la misma osición relativa de la webcam con respecto al marcador). El resto de elementos de la escena los ubicaremos en una posición relativa al origen de coordenadas ubicado en el centro del marcador.

Para empezar necesitamos una escena a la que pondremos una luz ambiental blanca y un fondo negro.

In [5]:
escena = pyrender.Scene(bg_color=(0,0,0), ambient_light=(1.0, 1.0, 1.0))

Para cada modelo que queramos incorporar a la escena necesitamos cargarlo y ubicarlo dentro de la escena mediante una matriz de transformación.

Para la carga y adaptación del modelo para que pueda ser usado por Pyrender empleamos trimesh.

In [6]:
nombrefi = "media/cubo.glb"
modelo_trimesh = trimesh.load(nombrefi, file_type='glb')
modelo_mesh = pyrender.Mesh.from_trimesh(list(modelo_trimesh.geometry.values()))

Para obtener la matriz de transformación que indica translación, rotación y escala del modelo empleamos mathutils partiendo de una matriz identidad y componiendo en orden las transformaciones necesarias. Hay que recordar que la composición de matrices no es una operación conmutativa por lo que el orden de composición es significativo. Lo normal es aplicar en primer lugar el escalado, después la rotación y finalmente la translación. Para compensar el cambio de sistema de coordenadas se puede aplicar una rotación de 90 grados en el eje X.

La matriz de transformación de translación la construimos mediante [Translation](https://docs.blender.org/api/blender_python_api_current/mathutils.html?highlight=translation#mathutils.Matrix.Translation) indicando las coordenadas X, Y y Z (expresadas en metros). Un modelo tiene su propio sistema de coordenadas local que suele tener su origen en el centro del modelo de modo que la translación que le aplicamos se traduce en la translación de su propio sistema de coordenadas local. Por ejemplo, si tenemos un modelo en forma de cubo de 10cm de lado (con su sistema de coordenadas local ubicado en su centro) y queremos situarlo sobre el centro del marcador, la matriz de transformación tendrá que situar su centro a 5cm de altura (eje Z).

In [7]:
mat_loc = mathutils.Matrix.Translation((0.0, 0.0, 0.05))

La matriz de transformación de rotación la construimos mediante [Rotation](https://docs.blender.org/api/blender_python_api_current/mathutils.html?highlight=rotation#mathutils.Matrix.Rotation), especificando el ángulo de rotación en radianes, el tamaño de la matriz (que en nuestro caso siempre será 4), y el eje alrededor del cual se realizará la rotación. Haremos una rotación de 90 grados en el eje X para que el modelo esté en la misma posición en la que fue definido (y así compensar el cambio de sistema de coordenadas). Si queremos otras rotaciones adicionales no tenemos más que ir definiéndolas para aplicarlas después en el orden adecuado.

**Nota**: mathutils permite expresar rataciones en forma de quaterniones pero ahora nos vamos a limitar al empleo tradicional de ángulos de Euler.

In [8]:
mat_rot = mathutils.Matrix.Rotation(math.radians(90.0), 4, 'X')

La matriz de transformación de escalado se contruye mediante [Scale](https://docs.blender.org/api/blender_python_api_current/mathutils.html?highlight=scale#mathutils.Matrix.Scale) indicando el factor de escala y el tamaño de la matriz de transformación que en nuestro caso siempre será 4. Un escalado de un factor 1 dejará el modelo en el tamaño original.

In [9]:
mat_sca = mathutils.Matrix.Scale(1.0, 4)

Finalmente la matriz de transformación final será fruto de la composición de las transformaciones indivicuales. Esta operación se aplica de derecha a izquierda por lo que la aplicación de scalado seguido de rotación seguido de translación se puede componer del siguiente modo:

In [10]:
meshpose = mat_loc @ mat_rot @ mat_sca

Los elementos se añaden a la escena en forma de nodos indicando la matriz de transformación que expresa su pose dentro de la escena.

In [11]:
modelo = pyrender.Node(mesh=modelo_mesh, matrix=meshpose) # Creamos un nodo indicando la malla y su pose
escena.add_node(modelo) # Y la añadimos a la escena

Tan solo nos falta en la escena la cámara. Pyrender permite definir cámaras ortogonales, en perspectiva y cámaras "personalizadas" a partir de un conjunto de parametros intrínsecos. Dado que el objetivo es mezclar de un modo adecuado mundo real y mundo virtual, necesitamos que el renderizado de la escena 3D se realice con las mismas características de la webcam. Por ello lo ideal es crear una cámara intrínseca con las mismas características que la webcam.

La cámara la crearemos usando [IntrinsicsCamera(fx, fy, cx, cy)](https://pyrender.readthedocs.io/en/latest/generated/pyrender.camera.IntrinsicsCamera.html), donde *fx* y *fy* son los [campos de visión (FOV)](https://es.wikipedia.org/wiki/Campo_de_visi%C3%B3n) y *cx* y *cy* indican las coordenadas del pixel central de la óptica. Estos parámetros podemos leerlos de la matriz característica de la cámara obtenida tras el proceso de calibrado.

In [12]:
fx = camara.cameraMatrix[0][0]
fy = camara.cameraMatrix[1][1]
cx = camara.cameraMatrix[0][2]
cy = camara.cameraMatrix[1][2]

camInt = pyrender.IntrinsicsCamera(fx, fy, cx, cy)
cam = pyrender.Node(camera=camInt)
escena.add_node(cam)

El nodo *cam* se ha añadido a la escena sin especificar su matriz de transformación (por defecto usará una matriz identidad). Esta matriz se irá actualizando en función de la pose detectada del marcador de Aruco.

Por último, Pyrender, además de ofrecer un visualizador de escenas 3D permite obtener las imágenes para ser procesadas de un modo independiente en lo que se conoce como [*screen rendering*](https://pyrender.readthedocs.io/en/latest/examples/offscreen.html#). Se trata de la función [OffscreenRenderer](https://pyrender.readthedocs.io/en/latest/generated/pyrender.offscreen.OffscreenRenderer.html#pyrender.offscreen.OffscreenRenderer) a la que hay que suministrar las dimensiones de la imagen (que haremos coincidir con las dimensiones de la imagen capturada por la cámara.

In [ ]:
camId = 0
bk = cuia.bestBackend(camId)
ar = cuia.myVideo(camId, bk)
hframe = ar.get(cv2.CAP_PROP_FRAME_HEIGHT)
wframe = ar.get(cv2.CAP_PROP_FRAME_WIDTH)
mirender = pyrender.OffscreenRenderer(wframe, hframe)

Una vez creado el renderizador, cada vez que queramos un render de la escena deberemos utilizar el método [render](https://pyrender.readthedocs.io/en/latest/generated/pyrender.offscreen.OffscreenRenderer.html#pyrender.offscreen.OffscreenRenderer.render). Esta función ofrece la imagen en color RGB resultado del render así como una matriz de profundidad que indica, para cada uno de los píxeles, la distancia a la que se encuentra de la cámara, con un valor negativo para identificar los píxeles del fondo. Esta matriz la usaremos para "recortar" la imagen que después será ubicada sobre la imagen de la cámara. Podemos implementar una función que realice este proceso.

In [14]:
def realidadMixta(renderizador, frame, escena):
    color, m = renderizador.render(escena)
    bgr = cv2.cvtColor(color, cv2.COLOR_RGB2BGR) #convertimos la imagen de color al espacio BGR

    _, m = cv2.threshold(m, 0, 1, cv2.THRESH_BINARY) #Umbralizamos la matriz de profundidad poniendo a cero los valores negativos y el resto a uno
    m = (m*255).astype(np.uint8) #Para usarla como canal alfa necesitamos expresarla en el rango [0,255] como números enteros
    m = np.stack((m,m,m), axis=2) #Creamos una imagen de 3 bandas repitiendo la máscara obtenida

    #A continuación empleamos la máscara y su inversa para combinar la imagen del frame con la imagen generada por el render
    inversa = cv2.bitwise_not(m)
    pp = cv2.bitwise_and(bgr, m)
    fondo = cv2.bitwise_and(frame, inversa)
    res = cv2.bitwise_or(fondo, pp)
    return(res)

Por último realizamos este proceso para cada frame capturado por la webcam en el que hayamos detectado el marcador adecuado. Implementaremos una función que devuelva las matrices de rotación y translación cuando se detecte un determinado marcador del que indicamos su tamaño.

In [15]:
diccionario = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_50)
detector = cv2.aruco.ArucoDetector(diccionario)

def detectarPose(frame, idMarcador, tam):
    bboxs, ids, rechazados = detector.detectMarkers(frame)
    if ids is not None:
        for i in range(len(ids)):
            if ids[i] == idMarcador:
                objPoints = np.array([[-tam/2.0, tam/2.0, 0.0],
                                      [tam/2.0, tam/2.0, 0.0],
                                      [tam/2.0, -tam/2.0, 0.0],
                                      [-tam/2.0, -tam/2.0, 0.0]])
                ret, rvec, tvec = cv2.solvePnP(objPoints, bboxs[i], camara.cameraMatrix, camara.distCoeffs)
                if ret:
                    return((True, rvec, tvec))
    return((False, None, None))

En definitiva el proceso que ha de realizarse para cada frame consta de los siguientes pasos:
* Detectar el marcador y obtener la pose (translación y rotación)
* Ubicar la cámara de la escena 3D en la posición de la webcam
* Combinar el renderizado del modelo 3D con la imagen de la webcam

In [16]:
def mostrarModelo(frame):
    ret, rvec, tvec = detectarPose(frame, 0, 0.19) #Buscaremos el marcador 0 impreso con 19cm de lado
    if ret:
        poseCamara = fromOpencvToPyrender(rvec, tvec) #Determinamos la posición de la cámara en forma de matriz de transformación de Pyrender
        escena.set_pose(cam, poseCamara) #Ubicamos la cámara en la posición obtenido
        frame = realidadMixta(mirender, frame, escena) #Mezclamos mundo real y mundo virtual
    return(frame)

In [17]:
ar.process = mostrarModelo
try:
    ar.play("AR", key=ord(' '))
finally:
    ar.release()

![Prueba de render](media/rendercubo.png "Prueba de render")